
# Clustering Text Documents using K-Means Algorithm


### This is an example showing how the scikit-learn can be used to cluster documents by topics using TF-IDF approach. 

This example uses a scipy.sparse matrix to store the features instead of standard numpy arrays.

Feature extraction method being used in this example:

  - **TfidfVectorizer** uses a in-memory vocabulary (a python dict) to map the most
    frequent words to features indices and hence compute a word occurrence
    frequency (sparse) matrix. The word frequencies are then reweighted using
    the Inverse Document Frequency (IDF) vector collected feature-wise over
    the corpus.


Two algorithms are demoed: ordinary k-means and its more scalable cousin
minibatch k-means.

Additionally, latent semantic analysis can also be used to reduce dimensionality and discover latent patterns in the data.

It can be noted that k-means (and minibatch k-means) are very sensitive to feature scaling and that in this case the IDF weighting helps improve the quality of the clustering by quite a lot as measured against the "ground truth" provided by the class label assignments of the 20 newsgroups dataset.

This improvement is not visible in the Silhouette Coefficient which is small for both as this measure seem to suffer from the phenomenon called "Concentration of Measure" or "Curse of Dimensionality" for high dimensional datasets such as text data. 

Other measures such as V-measure and Adjusted Rand Index are information theoretic based evaluation scores: as they are only based on cluster assignments rather than distances, hence not affected by the curse of dimensionality.

Note: as k-means is optimizing a non-convex objective function, it will likely end up in a local optimum. 
Several runs with independent random init might be necessary to get a good convergence.


## Import Libraries

In [53]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import make_pipeline
from sklearn import metrics

from time import time

import numpy as np

## Read Data

In [54]:
# #############################################################################

# Load some specific categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

# Uncomment the following to do the analysis on all the categories
# categories = None


In [55]:
print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()


Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
3387 documents
4 categories



In [56]:
labels = dataset.target
true_k = np.unique(labels).shape[0]

In [57]:
print("Extracting features from the training dataset using a sparse vectorizer")

n_features = 100

# Convert a collection of raw documents to a matrix of TF-IDF features.
vectorizer = TfidfVectorizer(max_df=0.5, max_features= n_features, min_df=2, stop_words='english', use_idf=True)
t0 = time()
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()


Extracting features from the training dataset using a sparse vectorizer
done in 1.875018s
n_samples: 3387, n_features: 100



In [58]:
X

<3387x100 sparse matrix of type '<class 'numpy.float64'>'
	with 48800 stored elements in Compressed Sparse Row format>

In [59]:
print("Performing Dimensionality Reduction using LSA")

n_components = 30

t0 = time()

# Vectorizer results are normalized, which makes KMeans behave as spherical k-means for better results. 
# Since LSA/SVD results are not normalized, we have to redo the normalization.
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)

lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

print()


Performing Dimensionality Reduction using LSA
done in 0.115999s
Explained variance of the SVD step: 53%



## Perform Clustering

In [60]:
# Whether to use Minibatch K-Means
minibatch=True
#minibatch=False

if minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1, init_size=1000, batch_size=1000, verbose=1)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, verbose=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)

print("Done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, km.labels_, sample_size=1000))


Clustering sparse data with MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
                init_size=1000, max_iter=100, max_no_improvement=10,
                n_clusters=4, n_init=1, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=1)
Init 1/1 with method: k-means++
Inertia for init 1/1: 713.872769
Minibatch iteration 1/400: mean batch inertia: 0.708108, ewa inertia: 0.708108 
Minibatch iteration 2/400: mean batch inertia: 0.697559, ewa inertia: 0.701880 
Minibatch iteration 3/400: mean batch inertia: 0.695015, ewa inertia: 0.697827 
Minibatch iteration 4/400: mean batch inertia: 0.689436, ewa inertia: 0.692874 
Minibatch iteration 5/400: mean batch inertia: 0.693373, ewa inertia: 0.693168 
Minibatch iteration 6/400: mean batch inertia: 0.691870, ewa inertia: 0.692402 
Minibatch iteration 7/400: mean batch inertia: 0.691571, ewa inertia: 0.691912 
Minibatch iteration 8/400: mean batch inertia: 0.687813, ewa inertia: 0.689492 
Miniba

In [61]:
print("Top terms per cluster:")

if n_components:
    original_space_centroids = svd.inverse_transform(km.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
else:
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :15]:
        print(' %s' % terms[ind], end='')
    print()

Top terms per cluster:
Cluster 0: university article posting uk just nntp host know think does don say like ac did
Cluster 1: space nasa access gov net article just like com cs posting host nntp earth think
Cluster 2: graphics university image file program files help posting host nntp need computer mail software format
Cluster 3: com god article people don jesus think say just know christian like sgi does believe
